## Library

In [1]:
import kagglehub
import os
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# import set_matplotlib_hangul # 한글 폰트 변환
import scipy.stats as stats
import re
from collections import Counter

/Users/mungughyeon/miniforge3/envs/Study/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Load
- 데이터 출처:  [Amazon Review Data](https://www.kaggle.com/datasets/tarkkaanko/amazon)

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

# Download latest version
path = kagglehub.dataset_download("tarkkaanko/amazon")
print(os.listdir(path)) # 다운로드된 파일 목록 확인

['amazon_reviews.csv']


In [3]:
warnings.filterwarnings("ignore", category=FutureWarning)

df = pd.read_csv(os.path.join(path, 'amazon_reviews.csv'), index_col=0)
# df.to_csv("../data/AB-Test.csv")
df

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4910,"ZM ""J""",1.0,I bought this Sandisk 16GB Class 10 to use wit...,2013-07-23,503,0,0,0,0,0.0,0.0
4911,Zo,5.0,Used this for extending the capabilities of my...,2013-08-22,473,0,0,0,0,0.0,0.0
4912,Z S Liske,5.0,Great card that is very fast and reliable. It ...,2014-03-31,252,0,0,0,0,0.0,0.0
4913,Z Taylor,5.0,Good amount of space for the stuff I want to d...,2013-09-16,448,0,0,0,0,0.0,0.0


## Data Summary

| Column                  | 설명 |
|------------------------|------------------------------------------------------------|
| `reviewerName`        | 리뷰 작성자의 이름 |
| `overall`             | 제품의 평점 (1~5점) |
| `reviewText`          | 사용자가 남긴 리뷰 텍스트 |
| `reviewTime`          | 리뷰가 작성된 날짜 |
| `day_diff`            | 기준일(데이터 수집일)과 리뷰 작성일 간의 차이 (일 단위) |
| `helpful_yes`         | 리뷰가 '도움이 됐다'고 표시된 횟수 |
| `helpful_no`          | 리뷰가 '도움이 안 됐다'고 표시된 횟수 |
| `total_vote`          | 해당 리뷰에 대한 총 투표 수 (`helpful_yes` + `helpful_no`) |
| `score_pos_neg_diff`  | 긍정적 투표(`helpful_yes`)에서 부정적 투표(`helpful_no`)를 뺀 값 |
| `score_average_rating`| `helpful_yes`를 총 투표 수로 나눈 평균 점수 |
| `wilson_lower_bound`  | 윌슨 신뢰 구간(Wilson Score) 기반으로 신뢰도를 반영한 순위 점수 |

## Data Preprocessing

In [4]:
# Check Data Type
def Check_Data_Type(df, return_data=False):
    df_str = df.select_dtypes(include=['object'])
    df_num = df.select_dtypes(include=['number'])

    print(f"Object Type Length: {df_str.shape[1]}")
    print(f"Num Type Length: {df_num.shape[1]}")

    if return_data: # 출력만 보려할 때
        return df_str, df_num

In [5]:
df.info()
print()
Check_Data_Type(df)

<class 'pandas.core.frame.DataFrame'>
Index: 4915 entries, 0 to 4914
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   reviewerName          4914 non-null   object 
 1   overall               4915 non-null   float64
 2   reviewText            4914 non-null   object 
 3   reviewTime            4915 non-null   object 
 4   day_diff              4915 non-null   int64  
 5   helpful_yes           4915 non-null   int64  
 6   helpful_no            4915 non-null   int64  
 7   total_vote            4915 non-null   int64  
 8   score_pos_neg_diff    4915 non-null   int64  
 9   score_average_rating  4915 non-null   float64
 10  wilson_lower_bound    4915 non-null   float64
dtypes: float64(3), int64(5), object(3)
memory usage: 460.8+ KB

Object Type Length: 3
Num Type Length: 8


In [6]:
# Check missing value
def Check_NaN(df):
    count = df.isna().sum() # 개수
    ratio = (count / len(df)) * 100 # 비율
    NaN_df = pd.DataFrame({'Count':count, 'Ratio(%)':ratio}).reset_index().rename(columns={'index':'Column'})
    NaN_df = NaN_df.sort_values(by=['Count'], ascending=False).reset_index(drop=True)

    return NaN_df

In [7]:
NaN_df = Check_NaN(df)
NaN_df

,Column,Count,Ratio(%)
0,reviewerName,1,0.020346
1,reviewText,1,0.020346
2,overall,0,0.000000
3,reviewTime,0,0.000000
4,day_diff,0,0.000000
5,helpful_yes,0,0.000000
6,helpful_no,0,0.000000
7,total_vote,0,0.000000
8,score_pos_neg_diff,0,0.000000
9,score_average_rating,0,0.000000


In [8]:
df.dropna(inplace=True) # drop NaN
df['reviewTime'] = pd.to_datetime(df['reviewTime'])
df['Year'] = df['reviewTime'].dt.year # year
df['Month'] = df['reviewTime'].dt.month # month
df['Day'] = df['reviewTime'].dt.day # day

# order column
order = ['reviewerName', 'overall', 'reviewText', 'reviewTime', 'Year', 'Month', 'Day', 'day_diff',
        'helpful_yes', 'helpful_no', 'total_vote', 'score_pos_neg_diff', 'score_average_rating', 'wilson_lower_bound']

df = df[order]
df

,reviewerName,overall,reviewText,reviewTime,Year,Month,Day,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,2013,10,25,409,0,0,0,0,0.0,0.0
2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,2012,12,23,715,0,0,0,0,0.0,0.0
3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,2013,11,21,382,0,0,0,0,0.0,0.0
4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,2013,7,13,513,0,0,0,0,0.0,0.0
5,2Cents!,5.0,It's mini storage. It doesn't do anything els...,2013-04-29,2013,4,29,588,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,"ZM ""J""",1.0,I bought this Sandisk 16GB Class 10 to use wit...,2013-07-23,2013,7,23,503,0,0,0,0,0.0,0.0
4911,Zo,5.0,Used this for extending the capabilities of my...,2013-08-22,2013,8,22,473,0,0,0,0,0.0,0.0
4912,Z S Liske,5.0,Great card that is very fast and reliable. It ...,2014-03-31,2014,3,31,252,0,0,0,0,0.0,0.0
4913,Z Taylor,5.0,Good amount of space for the stuff I want to d...,2013-09-16,2013,9,16,448,0,0,0,0,0.0,0.0


In [9]:
# Check duplication
du = df[df.duplicated(subset=['reviewerName'], keep='first')]
du

,reviewerName,overall,reviewText,reviewTime,Year,Month,Day,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
15,"Aaron ""Aaron""",5.0,I bought this to use with my go pro hero 3 bla...,2013-04-01,2013,4,1,616,0,0,0,0,0.0,0.0
20,Aaron,5.0,Works as expected. High transfer speed. Nice...,2013-10-27,2013,10,27,407,0,0,0,0,0.0,0.0
21,Aaron,5.0,Works great in a Samsung Galaxy S3. Formatted...,2013-12-29,2013,12,29,344,0,0,0,0,0.0,0.0
72,Adrian,5.0,does the job as advertised.if you need extra m...,2014-10-02,2014,10,2,67,0,0,0,0,0.0,0.0
109,Alan,4.0,SanDisk Ultra 32 GB MicroSDHC C10/UHS1 Memory ...,2014-01-31,2014,1,31,311,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,Tony,5.0,"It's a fast, affordable cheap microsd card. Wh...",2013-06-02,2013,6,2,554,0,0,0,0,0.0,0.0
4612,Tony,1.0,This may be the worst memory card I've ever pu...,2014-06-15,2014,6,15,176,0,0,0,0,0.0,0.0
4711,Veronica,3.0,This product works as described. It's just no...,2014-01-20,2014,1,20,322,0,0,0,0,0.0,0.0
4770,wayne,2.0,I bought two as a back up for my Gopro Hero 3....,2014-01-16,2014,1,16,326,0,0,0,0,0.0,0.0


In [10]:
# Check unique values
cols = ['reviewerName', 'Year', 'Month']

for col in df[cols].columns:
    print(f"[{col}] Unique Value: {df[col].unique()}")
    print(f"[{col}] Unique Length: {df[col].nunique()}\n")

[reviewerName] Unique Value: ['0mie' '1K3' '1m2' ... 'Z S Liske' 'Z Taylor' 'Zza']
[reviewerName] Unique Length: 4593

[Year] Unique Value: [2013 2012 2014]
[Year] Unique Length: 3

[Month] Unique Value: [10 12 11  7  4  3  2  5  1  8  9  6]
[Month] Unique Length: 12



## EDA

In [11]:
class Visualizer:

    # bar_plot 함수
    @staticmethod   
    def bar_plot(df, x_col, y_col, figsize=(10, 6), hue=None, palette="Blues_r", 
                 rotation=None, title=None, xlabel=None, ylabel=None, show_mean=False, mean_color='red', mean_linestyle='--'):
        plt.figure(figsize=figsize)
        
        if hue: # hue 지정
            sns.barplot(x=x_col, y=y_col, hue=hue, data=df, palette=palette)
        else: # hue 지정 X
            sns.barplot(x=x_col, y=y_col, data=df, palette=palette)

         # 평균선 추가 (옵션)
        if show_mean:
            mean_value = df[y_col].mean()  # y_col의 전체 평균값
            plt.axhline(mean_value, color=mean_color, linestyle=mean_linestyle, linewidth=1.5, label=f'Mean: {mean_value:.2f}')
            plt.legend()  # 범례 추가

        plt.xticks(rotation=rotation)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.show()

    # box_plot 함수
    @staticmethod
    def box_plot(df1, col, df2=None, figsize=(8, 6), color1='skyblue', color2='salmon', title=None):
        plt.figure(figsize=figsize)

        if df2 is None:
            # 단일 데이터프레임 boxplot
            sns.boxplot(y=df1[col], color=color1)  # 개별일 때는 color1 (기본값 skyblue)
            plt.ylabel(col)
            plt.title(title if title else f'{col} (Box Plot)')
        else:
            # 두 데이터프레임 비교 boxplot
            plt.subplot(1, 2, 1)
            sns.boxplot(y=df1[col], color=color1)  # 첫 번째 그룹은 color1
            plt.ylabel(col)
            plt.title(f'Group 1 - {col}')

            plt.subplot(1, 2, 2)
            sns.boxplot(y=df2[col], color=color2)  # 두 번째 그룹은 color2
            plt.ylabel(col)
            plt.title(f'Group 2 - {col}')

            plt.suptitle(title if title else f'{col} Comparison (Box Plot)', fontsize=12)

        plt.show()

    # hist_plot 함수
    @staticmethod
    def hist_plot(df1, col, df2=None, figsize=(8, 6), bins=30, alpha=0.6, color1='skyblue', color2='salmon', label1=None, label2=None, title=None):
        plt.figure(figsize=figsize)

        # 개별 데이터 프레임
        sns.histplot(df1[col], kde=True, bins=bins, alpha=alpha, color=color1, label=label1 if label1 else ('Group 1' if df2 is not None else None))
        
        if df2 is not None: # 두 그룹 비교
            sns.histplot(df2[col], kde=True, bins=bins, alpha=alpha, color=color2, label=label2 if label2 else 'Group 2')
            plt.legend()  # 두 그룹일 때만 범례 표시

        plt.title(title if title else f'{col} Distribution (Histogram + KDE)')
        plt.xlabel(col)
        plt.show()

    # line_plot 함수
    @staticmethod
    def line_plot(df, x_col, y_col, hue=None, figsize=(10, 6), palette=None, marker='o', linewidth=2, rotation=None, title=None, xlabel=None, ylabel=None):
        plt.figure(figsize=figsize)
        
        # hue가 None이면 일반 lineplot, hue가 있으면 그룹별 lineplot
        sns.lineplot(data=df, x=x_col, y=y_col, hue=hue, palette=palette, marker=marker, linewidth=linewidth)
        plt.xticks(rotation=rotation)  
        plt.title(title if title else f'{y_col} by {x_col} (Line Plot)')  
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.grid(True)
        if hue: # hue가 있을 경우 범례 추가
            plt.legend(title=hue)
        plt.show()

    # heatmap 함수
    @staticmethod
    def heatmap(df, figsize=(10, 6), target_col=None, threshold=0.4, annot=True, fmt='.2f', cmap='Blues', annot_kws={"size": 12}, cbar=True, title=None):
        corr = df.select_dtypes(include=['number']).corr() # 숫자형 데이터 타입

        if target_col: # 타겟 컬럼과의 상관관계만 추출
            corr = corr[[target_col]].sort_values(by=target_col, ascending=False)
            corr = corr[abs(corr[target_col]) >= threshold].dropna()
            
            title = title or f"'{target_col}'과의 상관관계 (≥ {threshold})"
            plt.figure(figsize=(4, len(corr) * 0.6))
            sns.heatmap(corr.T, annot=annot, fmt=fmt, cmap=cmap, annot_kws=annot_kws, cbar=cbar)
        
        else: # 전체 상관관계 매트릭스
            title = title or "전체 상관관계 히트맵"
            plt.figure(figsize=figsize)
            sns.heatmap(corr, annot=annot, fmt=fmt, cmap=cmap, annot_kws=annot_kws, cbar=cbar)

        plt.title(title)
        plt.show()

In [12]:
# groupby 함수
def group_by(df, target_col_1, sub_col, target_col_2=None, function=None, rename_col=None):
    # 단일 컬럼 & 다중 컬럼
    group_cols = [target_col_1] if target_col_2 is None else [target_col_1, target_col_2]
    if isinstance(sub_col, str):
        sub_col = [sub_col]

    if function is None:
        function = ['count', 'sum', 'mean', 'min', 'max']
    elif isinstance(function, str):  
        function = [function]  # 단일 문자열 입력 시 리스트로 변환

    df_group = df.groupby(group_cols)[sub_col].agg(function).reset_index()

    # 컬럼명 지정
    if rename_col:
        if isinstance(rename_col, str):  # 단일 컬럼명 변경
            df_group.columns = group_cols + [rename_col]
        elif isinstance(rename_col, list) and len(rename_col) == len(sub_col):  # 여러 개 컬럼명 변경
            df_group.columns = group_cols + rename_col
    else:
        df_group.columns = group_cols + sub_col # 기본 컬럼명 유지
    return df_group

# value_count 함수
def value_cnt(df, target_col):
    value_cnt = df[target_col].value_counts().reset_index()
    return value_cnt

In [13]:
df

,reviewerName,overall,reviewText,reviewTime,Year,Month,Day,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,2013,10,25,409,0,0,0,0,0.0,0.0
2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,2012,12,23,715,0,0,0,0,0.0,0.0
3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,2013,11,21,382,0,0,0,0,0.0,0.0
4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,2013,7,13,513,0,0,0,0,0.0,0.0
5,2Cents!,5.0,It's mini storage. It doesn't do anything els...,2013-04-29,2013,4,29,588,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,"ZM ""J""",1.0,I bought this Sandisk 16GB Class 10 to use wit...,2013-07-23,2013,7,23,503,0,0,0,0,0.0,0.0
4911,Zo,5.0,Used this for extending the capabilities of my...,2013-08-22,2013,8,22,473,0,0,0,0,0.0,0.0
4912,Z S Liske,5.0,Great card that is very fast and reliable. It ...,2014-03-31,2014,3,31,252,0,0,0,0,0.0,0.0
4913,Z Taylor,5.0,Good amount of space for the stuff I want to d...,2013-09-16,2013,9,16,448,0,0,0,0,0.0,0.0
